In [17]:
def train(data):
    
    sentences = data.split('\n') #Разделим нашу строку на предложения
    tokenized_corpus = []
    for sentence in sentences:
        tokenized_sentense = sentence.split()
        tokenized_corpus.append(tokenized_sentense)
    # На этом этапе в tokenized_corpus лежит массив предложений, которые представляют собой массив слов
    
    vocabulary = {} 
    index = 0
    for sentense in tokenized_corpus:
        for token in sentense:
            if token not in vocabulary:
                vocabulary[token] = index
                index += 1
    vocabulary_size = len(vocabulary)
    #vocabulary - словарь вида (Слово - индекс). Нам понадобится словарь вида (индекс - слово) - ind_word
    ind_word = {v:k for k, v in vocabulary.items()}
    
    window_size = 2
    pairs = []
    
    #Создадим пары слов (центральное слово - контекстное слово)
    for sentense in tokenized_corpus:
        indices = []
        for word in sentense:
            indices.append(vocabulary[word])
        
        for mid_pos in range(len(indices)):
            for stride in range(-window_size, window_size + 1):
                    context_pos = mid_pos + stride
                
                    if context_pos < 0 or context_pos >= len(indices) or context_pos == mid_pos:
                        continue
                    
                    context_index = indices[context_pos]
                
                    pairs.append((indices[mid_pos], context_index))
      # В pairs в итоге лежат индексы пар контекстно-связанных слов
    
  #Создадим функцию, которая возвращает one-hot вектор слова
    def get_input_layer(word_idx):
        x = torch.zeros(vocabulary_size).float()
        x[word_idx] = 1.0
        return x
    
    #Не забудем импортировать необходимые библиотеки

    import numpy as np
    import torch
    from torch.autograd import Variable
    import torch.nn.functional as F
    
    #Делаем training_loop
    embedding_dims = 50
    W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
    W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
    num_epochs = 100
    learning_rate = 0.001

    for epo in range(num_epochs):
        loss_val = 0
        for data, target in pairs:
            x = Variable(get_input_layer(data)).float()
            y_true = Variable(torch.from_numpy(np.array([target])).long())

            z1 = torch.matmul(W1, x)
            z2 = torch.matmul(W2, z1)
    
            log_softmax = F.log_softmax(z2, dim=0)

            loss = F.nll_loss(log_softmax.view(1,-1), y_true)
            loss_val += loss.data
            loss.backward()
            W1.data -= learning_rate * W1.grad.data
            W2.data -= learning_rate * W2.grad.data

            W1.grad.data.zero_()
            W2.grad.data.zero_()
            
    #Натренировали матрицы весов, теперь делаем словарь вида (слово - векторное представление слова)
    w2v_dict = {}
    for word in vocabulary:
        x = get_input_layer(vocabulary[word]).float()
        z1 = torch.matmul(W1, x)
        word_vector = z1.detach().numpy()
        w2v_dict[word] = word_vector

    return w2v_dict






In [18]:
corpus = 'for many years I have been\n waiting for a human \n who can beat the dragon \n in this case i will marry\n this breathtaking knight '

In [19]:
train(corpus)

{'for': array([-0.755719  ,  0.21397416, -0.2836862 ,  0.75444376, -0.1890428 ,
        -0.54649067, -0.84183633,  0.7414315 ,  1.1090454 ,  2.2869356 ,
         1.9038846 , -0.50699455,  0.29765633,  0.860487  ,  0.6462936 ,
         0.6537008 , -1.4610603 ,  0.15299419, -0.11193819,  0.9393167 ,
        -1.0530921 , -0.8897935 ,  0.390139  ,  1.0911968 ,  1.111948  ,
        -0.25119978,  0.52596873,  0.7092808 , -0.9645561 , -0.24534468,
         0.3067979 ,  1.2586058 ,  0.46165246,  0.15773448,  1.7173713 ,
        -0.83308023,  2.4041724 , -0.44643778, -0.5580194 ,  0.94662726,
        -2.468372  , -1.6508702 , -1.2321093 , -0.6628716 ,  1.1518673 ,
         0.09430294, -0.11226371, -1.9671468 ,  0.41909385, -0.7233213 ],
       dtype=float32),
 'many': array([ 0.8387469 ,  0.5930457 ,  0.1936006 ,  0.41546637, -1.3903667 ,
        -1.1279942 , -0.47481352, -0.52557385, -2.102289  , -0.36576718,
         1.3459631 ,  1.2586352 ,  0.8819854 , -0.78009814, -0.59106296,
        -0.2